In [ ]:
# start coding here

import anndata
import random
import json
random.seed(snakemake.params.seed)

In [ ]:
def convert_datapoint(key, annotation):
    question = random.choice(snakemake.params.questions)
    # for each of them choose whether it's on the start or end
    if bool(random.getrandbits(1)):
        question = f"{snakemake.params.transcriptome_tag}\n{question}"
    else:
        question = f"{question}\n{snakemake.params.transcriptome_tag}"

    return {
        "id": key,
        "image": key,
        "conversations": [
            {
                "from": "human",
                "value": question,
            },
            {
                "from": "gpt",
                "value": annotation,
            }
        ]
    }
    

In [ ]:
with open(snakemake.input.annotations_archs4_geo) as f:
    annotations = json.load(f)

annotations = {key: value_list[snakemake.params.annotation_replicate] for key, value_list in annotations.items()}

In [ ]:
with open(snakemake.input.annotations_cellxgene_census) as f:
    annotations.update(json.load(f))

In [ ]:
dataset = [convert_datapoint(key, annotation) for key, annotation in annotations.items()]

with open(snakemake.output.train_set, "w") as f:
    json.dump([d for d in dataset if d["id"] not in snakemake.params.test_ids], f, indent=4)

with open(snakemake.output.test_set, "w") as f:
    json.dump([d for d in dataset if d["id"] in snakemake.params.test_ids], f, indent=4)